In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score,cross_val_predict,KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import log_loss
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os


# adjusting the random_state of train_test_split
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size=0.2,random_state=42)
rfr = RandomForestRegressor(random_state=40)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
CV_score = cross_val_score(rfr, Xtrain, Ytrain, cv=kf).mean()
regressor = rfr.fit(Xtrain, Ytrain)
score_test = regressor.score(Xtest,Ytest)
print("5cv:",CV_score,"TEST:",score_test)
score_max = 0
imax = 0
for i in range(300):     
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(x1,y1,test_size=0.2,random_state=i)
    rfr = RandomForestRegressor(random_state=40)
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    CV_score = cross_val_score(rfr, Xtrain, Ytrain, cv=kf).mean()
    regressor = rfr.fit(Xtrain, Ytrain)
    test_predictions = regressor.predict(Xtest)
    score_test = regressor.score(Xtest,Ytest)
    mae_test = MAE(Ytest, test_predictions)
# select the group with better evaluation metrics
    if CV_score>0.6 and score_test>0.6 and (score_test-CV_score<=0.1):
        if score_max < score_test:
            score_max = score_test
            imax = i
            print("5cv:",CV_score,"TEST:",score_test,"random_state",i,'mae_test', mae_test)


#Optimize the hyperparameters
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size=0.2,random_state=imax)
score_5cv_all = []
for i in range(0, 200, 1):
    rfr =RandomForestRegressor(random_state=i)
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    score_5cv =cross_val_score(rfr, Xtrain, Ytrain, cv=kf).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
random_state_5cv = range(0, 200)[score_5cv_all.index(max(score_5cv_all))]

print("Best_5cv score：{}".format(score_max_5cv),
      "random_5cv:{}".format(random_state_5cv))


score_5cv_all = []
for i in range(1, 200, 1):
    rfr = RandomForestRegressor(n_estimators=i
                                ,random_state=random_state_5cv)
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    score_5cv =cross_val_score(rfr, Xtrain, Ytrain, cv=kf).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
n_est_5cv = range(1,300)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "n_est_5cv:{}".format(n_est_5cv))


score_5cv_all = []
for i in range(1, 100, 1):
    rfr = RandomForestRegressor(n_estimators=n_est_5cv
                                ,random_state=random_state_5cv
                                ,max_depth=i)
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    score_5cv =cross_val_score(rfr, Xtrain, Ytrain, cv=kf).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
max_depth_5cv = range(1,100)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_depth_5cv:{}".format(max_depth_5cv))


score_5cv_all = []
for i in range(1,Xtrain.shape[1]+1):
    rfr = RandomForestRegressor(n_estimators=n_est_5cv
                                ,random_state=random_state_5cv
                                ,max_depth=max_depth_5cv
                                ,max_features=i)
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    score_5cv =cross_val_score(rfr, Xtrain, Ytrain, cv=kf).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
max_features_5cv = range(1, Xtrain.shape[1]+1)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_features_5cv:{}".format(max_features_5cv))



Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size=0.2,random_state=imax)
rfr = RandomForestRegressor(n_estimators=n_est_5cv
                            ,random_state=random_state_5cv
                            ,max_depth=max_depth_5cv
                            ,max_features=max_features_5cv)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
score_5cv =cross_val_score(rfr, Xtrain, Ytrain, cv=kf).mean()
CV_predictions = cross_val_predict(rfr, Xtrain, Ytrain, cv=5)
mae_CV = MAE(Ytrain, CV_predictions)
rmse_CV = np.sqrt(MSE(Ytrain,CV_predictions))
print("5cv:",score_5cv)
print("5CV_MAE",mae_CV)
print("rmse_5CV",rmse_CV)

regressor = rfr.fit(Xtrain, Ytrain)
test_predictions = regressor.predict(Xtest)
score_test = regressor.score(Xtest,Ytest)
rmse_test = np.sqrt(MSE(Ytest,test_predictions))
mae_test = MAE(Ytest, test_predictions)
print("test:",score_test)
print('mae_test', mae_test)